# FlexV

## Version history

Version: 0.1 (test) 

# Segmentation

In [ ]:
import numpy as np
import cv2
import os
import time

#nameOfVid = 'WL400_064-low-res.mp4'            #this works
nameOfVid = 'WL400_0V5.mp4'                          #name of video to be analysed


''' Sets modified dimensions for videoframe '''
def rescale_frame(frame,percent=75):         #from:https://www.youtube.com/watch?v=y76C3P20rwc
    scale_percent = percent
    width=int(frame.shape[1]*scale_percent/100)
    height=int(frame.shape[0]*scale_percent/100)               
    dim = (width,height)
    return cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)

cap = cv2.VideoCapture(nameOfVid)           #load video

''' Display some basic properties of the movie '''
width = cap.get(3)
height = cap.get(4)
totalf = cap.get(cv2.CAP_PROP_FRAME_COUNT)  #total number of frames
print ('Name of video to be analysed: ', nameOfVid)
print ('Width: ', width, 'pixel')
print ('Height: ', height, 'pixel')
print ('Total number of frames: ', totalf)


while(cap.isOpened()):  # check !
    ret, frame = cap.read()                     # capture frame-by-frame
    if ret: 
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)                                #Convert to Grayscale
        ret,simpleThreshold = cv2.threshold(gray, 140, 250, cv2.THRESH_BINARY)        #Simple threshold
        simpleThreshold=rescale_frame(simpleThreshold, percent = 50)                  #rescale frame
        cv2.imshow('Thresholding Method used',simpleThreshold)                        #open thresholded video in new window
    if cv2.waitKey(1) & 0xFF == ord('q'):    #quit by hitting 'q' in movie
        break
        
# When everything is done release the capture
cap.release()
cv2.destroyAllWindows()        

Name of video to be analysed:  WL400_0V5.mp4
Width:  1920.0
Height:  1440.0
Total number of frames:  3597.0


## Filtering 

## step 5 - the main algorithm

In [5]:
# -*- coding: utf-8 -*-
"""
findNextPoint created by Moritz Thom 2018
"""

import numpy as np
    
def findNextPoint(sX, sY, sAngle, dist, thresholding, dAngleInc, limAngle, width, height):
    '''
    This function searches for black pixels at defined distances and if found return the position
    
    Input:
    
    sX - X coordinate of starting point
    sY - Y coordinate of starting point
    sAngle - Angle (in degree) to start searching 
    dist - distance (in pixels) at which search is being conducted    
    thresholding - thresholding method
    dAngleInc - angle increment (in degrees)
    limAngle - limit of search angle (in degrees)
    width - width of each frame (in px)
    height - height of each frame (in px)
        
    Output:
    
    fP - reports if black pixel is found (= 1)  
    eX - X coordinate of black pixel found
    eY - Y coordinate of black pixel found
    eAngle - angle at which the black pixel was found 
    l - absolute length to black pixel found
        
    '''
    i = 0
    dAngle = 0

    while dAngle <= limAngle and dAngle >= -limAngle :          #modify this for more/less accuracy 
        if i != 0 and i % 2 == 0 :
            dAngle = dAngle + dAngleInc
        #switch from negative to positive at each iteration:
        if i % 2 == 0 :
            absdAngle = -dAngle
        else :
            absdAngle = dAngle
        out = 0 #indicates wether the resulting vector is out of bounds (=1)        
        eAngleRad = (sAngle + absdAngle)* np.pi / 180  #convert to radiant
        #calculate (and round) the dist in px in x,Y direction:
        dX = np.sin(eAngleRad) * dist 
        dY = np.cos(eAngleRad) * dist
        dX = np.around(dX) #round x
        dY = np.around(dY) #round y
        
        l = np.sqrt(dX*dX + dY*dY)                  #calcluate the absolute l
        chkX = int(sX - dX)
        chkY = int(sY - dY)
        eAngle = eAngleRad * 180 / np.pi            #transform into degrees    
        #Check here if out of bounds:
        if chkX > width or chkX < 0:
            print ('chkX: ',chkX, ' Width: ', width, 'Out of bounds')
            out = 1
            break
        if chkY > height or chkX < 0:
            print ('chkY: ',chkY, ' Height: ', height, 'Out of bounds')
            out = 1
            break
        if out == 0:
            if thresholding[chkY, chkX] == 0:
                eX = chkX
                eY = chkY
                fP = 1
                return fP, eX, eY, eAngle, l
        fP = 0
        i = i + 1
    fP = 0
    eX = 0
    eY = 0
    eAngle = 0
    l = 0
    return fP, eX, eY, eAngle, l
        

## step 1:

In [7]:
#Preferences:
vers = 0.1                                        #version nr.
nameOfVid = 'WL400_064-low-res.mp4'            #this works
#nameOfVid = 'WL400_0V5.mp4'                          #name of video to be analysed
saveResults = 1                         
writeResults = 1
showthresholdMov = 1
showOrigTrack = 1

#Variables
xCollector = []                                 #container
yCollector = []                                 #container
fnr = 0                                         #frame number
        



#new
def rescale_frame(frame,percent=75):         #from:https://www.youtube.com/watch?v=y76C3P20rwc
    scale_percent = percent
    width=int(frame.shape[1]*scale_percent/100)
    height=int(frame.shape[0]*scale_percent/100)               
    dim = (width,height)
    return cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)
    



'''
Main Program:
'''        

print ('This is pyFlexCV Version:', vers)
print ('written by Moritz Thom')
cap = cv2.VideoCapture(nameOfVid)

#Some basic properties of the movie
width = cap.get(3)
height = cap.get(4)
print ('Name of video to be analysed: ', nameOfVid)
print ('Width: ', width)
print ('Height: ', height)

while(cap.isOpened()):  # check !
    start_time = time.time()
    totalf = cap.get(cv2.CAP_PROP_FRAME_COUNT)  #total number of frames    
    fnr = fnr + 1                               #framenr    
    ret, frame = cap.read()                     # capture frame-by-frame
    
    if ret: 
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #Convert to Grayscale
        #ret,simpleThreshold = cv2.threshold(gray, 90, 250, cv2.THRESH_BINARY) #Simple threshold initial
        ret,simpleThreshold = cv2.threshold(gray, 140, 250, cv2.THRESH_BINARY) #Simple threshold, test1
        
        
#==============================================================================    
# Algorithm to analyse flexi veg
# based on function findNextPoint   
#==============================================================================        
        #Initial values (modify if needed, see documentation of findNextPoint.py for further details):
        sX = 445
        sY = 465
        sAngle = 0
        dist = 10 # in Pixels
        end = 0
        thresholding = simpleThreshold 
        dAngleInc = 0.1
        limAngle = 90
        distInc = 0
        maxIter = 11        #will not work with 25 as out of bounds
        x = [] #create an empty list where x values will be saved
        y = []
        
        #loop over one plant leaf
        while distInc <= maxIter:
            #print ('New starting point!')
            #print (distInc)
            fP, eX, eY, eAngle, l = findNextPoint(sX, sY, sAngle, dist, thresholding, dAngleInc, limAngle, width, height)
            if fP == 1:
                #cv2.line(simpleThreshold, (sX,sY),(eX,eY), 0, 5)
                cv2.line(frame, (sX,sY),(eX,eY), (0,255,0), 2) #print line    
                sX = eX
                sY = eY
                sAngle = eAngle         
                if saveResults == 1:       
                    x.append(sX)
                    y.append(sY)
                    
            if fP == 0:
                dist = dist + distInc               #here search further 
                distInc = distInc +1 
            
            if distInc == maxIter and saveResults == 1:     #save data to container
                xCollector.append(x)
                yCollector.append(y)
        #rescale frame !!!!new
        frame=rescale_frame(frame, percent = 30)
        thresholding=rescale_frame(thresholding, percent = 30)
        #for comparison display the thresholding method used above
        if showthresholdMov == 1:
            cv2.imshow('Thresholding Method used',thresholding)
        if showOrigTrack == 1:
            cv2.imshow('Original with tracking',frame)
        
        #Some printing to console:        
        elapsed_time = time.time() - start_time
        print ('Processed frame: ', fnr, '/', "%.0f" % totalf,' time/frame: ', "%.2f" % elapsed_time,' s')
      
      
    if cv2.waitKey(1) & 0xFF == ord('q'):    #quit by hitting 'q' in movie
        break
    
    
print ('Processing complete')
    
#==============================================================================    
#write results    
#==============================================================================

if writeResults == 1:
    np.savetxt('x.tmp', xCollector, delimiter=",", fmt='%s')
    np.savetxt('y.tmp', yCollector, delimiter=",", fmt='%s')
    
    #do some formatting for nice Matlab readability:
    i = 0
    with open('x.tmp') as f:
        lines = f.readlines()
    
    
    while i < len(lines):
        #read line 1:
            s = lines[i]
            #truncate:
            s = s[1:-2]
            s += '\n'
            with open('x.dat','a') as g:          #this appends text
                g.write(s)
                i=i+1
                
    #replace brackets in Y
    i = 0

    with open('y.tmp') as f:
        lines = f.readlines()
    
    
    while i < len(lines):
        #read line 1:
            s = lines[i]
            #truncate:
            s = s[1:-2]
            s += '\n'
            with open('y.dat','a') as g:          #this appends text
                g.write(s)
                i=i+1
    
    #delete temporary files:
    os.remove('x.tmp')
    os.remove('y.tmp')

    print('Writing results complete')    

# When everything is done release the capture
cap.release()
cv2.destroyAllWindows()

This is pyFlexCV Version: 0.1
written by Moritz Thom
Name of video to be analysed:  WL400_064-low-res.mp4
Width:  700.0
Height:  720.0
Processed frame:  1 / 301  time/frame:  1.08  s
Processed frame:  2 / 301  time/frame:  0.64  s
Processed frame:  3 / 301  time/frame:  0.61  s
Processed frame:  4 / 301  time/frame:  0.63  s
Processed frame:  5 / 301  time/frame:  0.62  s
Processed frame:  6 / 301  time/frame:  0.61  s
Processed frame:  7 / 301  time/frame:  0.60  s
Processed frame:  8 / 301  time/frame:  0.63  s
Processed frame:  9 / 301  time/frame:  0.61  s
Processed frame:  10 / 301  time/frame:  0.61  s
Processed frame:  11 / 301  time/frame:  0.61  s
Processed frame:  12 / 301  time/frame:  0.60  s
Processed frame:  13 / 301  time/frame:  0.60  s
Processed frame:  14 / 301  time/frame:  0.60  s
Processed frame:  15 / 301  time/frame:  0.60  s
Processed frame:  16 / 301  time/frame:  0.60  s
Processed frame:  17 / 301  time/frame:  0.61  s
Processed frame:  18 / 301  time/frame:  

# watch the original video file here

<video controls src="WL400_064-low-res.mp4" />

# test widgets
https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html

In [10]:
import ipywidgets as widgets

In [16]:
w = widgets.IntSlider()
w.min = 100
w.max = 200
display(w)

IntSlider(value=100, max=200, min=100)

In [15]:
w.value

100

### doesnt work

In [39]:
widgets.Text(
        value='Hello World',
        placeholder='Type something',
        description='String:',
        disabled=False
)

Text(value='Hello World', description='String:', placeholder='Type something')

### works:

In [41]:
w = widgets.Text(value='Hello World',description='String:',)
display(w)

Text(value='Hello World', description='String:')

In [37]:
w.value

'test'